In [26]:
import gym
import ray

In [27]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [47]:
# Initialize Ray
ray.shutdown()

In [48]:
from rayflow import Rayflow
r = Rayflow.load("test_input.yml", minio_volume_path="./minio_volume/", mlflow_volume_path="./mlflow_volume/",no_ray=True)

HOST INFO {'192.168.1.65': {'ansible_python_interpreter': '~/anaconda3/envs/py37/bin/python', 'ansible_connection': 'local'}}


In [49]:
r.start()

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-ke

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-976122t1xy_js/tmpcpei9ko8 TO /home/shyam/.ansible/tmp/ansible-tmp-1601342872.6787903-157488932188264/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601342872.6787903-157488932188264/ /home/shyam/.ansible/tmp/ansible-tmp-1601342872.6787903-157488932188264/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601342872.6787903-157488932188264/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601342872.6787903-157488932188264/ > /dev/null 2>&1 && sleep 0'
ok: [192.168.1.65] => {
    "changed": false,
    "invocation": {
        "module_args": {
            "api_version": "auto",
    

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-97807cvu8ivbh/tmpmfmhmclv TO /home/shyam/.ansible/tmp/ansible-tmp-1601342876.8592896-32761953997609/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601342876.8592896-32761953997609/ /home/shyam/.ansible/tmp/ansible-tmp-1601342876.8592896-32761953997609/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601342876.8592896-32761953997609/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601342876.8592896-32761953997609/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
    

<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601342877.631243-27241240586826/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
            "Args": [
                "server",
                "--backend-store-uri",
                "/mlflow",
                "--host",
                "192.168.1.65",
                "-p",
                "5000",
                "--default-artifact-root",
                "s3://artifacts/"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "mlflow",
                    "server",
                    "--backend-store-uri",
                    "/mlflow",
                    "--host",
                    "192.168.1.65",
                    "-p",
                    "5000",

In [50]:
ray.init(ignore_reinit_error=True,num_cpus=12, num_gpus=1)

2020-09-28 18:28:43,893	INFO resource_spec.py:231 -- Starting Ray with 31.84 GiB memory available for workers and up to 15.92 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-28 18:28:44,325	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2020-09-28_18-28-43_892160_91413/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-28_18-28-43_892160_91413/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-28_18-28-43_892160_91413'}

In [51]:
from minerl_rllib.envs import register
register()

In [70]:
from ray.rllib.agents.ppo import PPOTrainer

train_name = "mlflow_train"
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"DUDE",
    "create_new_experiment": True
}
trainer_config = {
        "env": "MineRLNavigateDenseVectorObf-v0",
         "num_gpus": 1.0,
         "num_workers": 1,
         "num_envs_per_worker": 1,
         "num_gpus_per_worker":1,
         "num_cpus_per_worker":5,
         "eager": False,
         "use_pytorch": True,
         'monitor':True, 
}
saver_config = {
        "checkpoint_on_end": True,
        "checkpoint_freq": 2
}
stop = {
        "training_iteration": 3,
}
tune_args = {
        "checkpoint_at_end": True,
        "checkpoint_freq": 2,
        "resources_per_trial":{'gpu': 1, 'cpu':5}
}

config = {
    "trainer_config":trainer_config,
    "logging_config": mlflow_logging_config
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
from rayflow.tune import Tune
t = Tune()

In [58]:
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer

In [59]:
run_id, output_config, experiment_analysis = t.run_training(
               trainer=PPOTrainer, 
               name=train_name, 
               stop=stop, 
               mlflow_logging_config=mlflow_logging_config, 
               trainer_config=trainer_config,
               saver_config=saver_config,
               tune_args=tune_args,
               )

Trial name,status,loc
MLFlowTrainer_2866f_00000,RUNNING,


(pid=98386) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/utils/util.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
(pid=98386)   from collections import defaultdict, deque, Mapping, Sequence
(pid=98386) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/tensorflow/python/autograph/utils/testing.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=98386)   import imp
(pid=98386) WARNING:tensorflow:From /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=98386) Instructions for updating:
(pid=98386) non-resource variables are not supported in the long term
(pid=98386) /home/shyam/a

Result for MLFlowTrainer_2866f_00000:
  custom_metrics: {}
  date: 2020-09-28_18-30-45
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 2e32b3ddd9964b0d84ab9bfff2a9a03b
  experiment_tag: '0'
  hostname: kokkgoblin
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.85855746269226
        entropy_coeff: 0.0
        kl: 0.0200020294287242
        policy_loss: -0.023124374507460743
        total_loss: 0.2245779470540583
        vf_explained_var: -0.0003417879343032837
        vf_loss: 0.2437019175849855
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.1.65
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 19.946666666666665
    gpu_util_percent0: 0.1645
    ram_util_percent: 27.856

Trial name,status,loc,iter,total time (s),ts,reward
MLFlowTrainer_2866f_00000,RUNNING,192.168.1.65:98386,1,95.4444,4000,nan


(pid=98385) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/minerl/herobraine/wrappers/obfuscation_wrapper.py:67: RuntimeWarning: invalid value encountered in true_divide
(pid=98385)   x[..., a:b] = e_x / e_x.sum(axis=-1)


...

Result for MLFlowTrainer_2866f_00000:
  custom_metrics: {}
  date: 2020-09-28_18-32-59
  done: true
  episode_len_mean: 6000.0
  episode_reward_max: 9.359046936035156
  episode_reward_mean: 8.491580963134766
  episode_reward_min: 7.624114990234375
  episodes_this_iter: 1
  episodes_total: 2
  experiment_id: 2e32b3ddd9964b0d84ab9bfff2a9a03b
  experiment_tag: '0'
  hostname: kokkgoblin
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.45000000000000007
        cur_lr: 5.0e-05
        entropy: 90.9316759109497
        entropy_coeff: 0.0
        kl: 0.01374838498304598
        policy_loss: -0.021691187284886837
        total_loss: 0.09964861010666937
        vf_explained_var: -1.862645149230957e-09
        vf_loss: 0.11515302280895412
    num_steps_sampled: 12000
    num_steps_trained: 12000
  iterations_since_restore: 3
  node_ip: 192.168.1.65
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 16.26463414634146

Trial name,status,loc,iter,total time (s),ts,reward
MLFlowTrainer_2866f_00000,RUNNING,192.168.1.65:98386,3,227.142,12000,8.49158


Trial name,status,loc,iter,total time (s),ts,reward
MLFlowTrainer_2866f_00000,TERMINATED,,3,227.142,12000,8.49158


Trial name,status,loc,iter,total time (s),ts,reward
MLFlowTrainer_2866f_00000,TERMINATED,,3,227.142,12000,8.49158


(pid=raylet) E0928 18:34:01.431208 98357 98357 process.cc:434] Failed to kill process 98385 with error system:3: No such process


In [65]:
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"DUDE",
    "create_new_experiment": False
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
config = {
    "trainer_config":  trainer_config,
    "logging_config": mlflow_logging_config,
    "trainer": PPOTrainer,
}

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
mlflow_trainer = t.restore_trainer(PPOTrainer, trainer_config, mlflow_logging_config, run_id=run_id, checkpoint_path="checkpoint_3", artifact_dir="./mlflow_artifacts/")

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-09-28 18:51:19,720	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
2020-09-28 18:51:19,721	WARNING deprecation.py:30 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.wa

In [69]:
mlflow_trainer

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
r.stop()

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601342845.2188673-106492455120504/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "changed": true,
    "invocation": {
        "module_args": {
            "api_version": "auto",
            "auto_remove": false,
            "blkio_weight": null,
            "ca_cert": null,
            "cap_drop": null,
            "capabilities": null,
            "cleanup": false,
            "client_cert": null,
            "client_key": null,
            "command": null,
            "comparisons": null,
            "cpu_period": null,
            "cpu_quota": null,
            "cpu_shares": null,
            "cpuset_cpus": null,
            "cpuset_mems": null,
            "debug": false,
            "detach": false,
            "device_read_bps": null,
            "device_read_iops": null,
            "device_write_bps": null,
            "device_write_iops": null,
            "devices": null,
          

2020-09-28 18:27:31,344	INFO (unknown file):0 -- gc.collect() freed 900 refs in 0.13464439000017592 seconds


<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601342846.4768205-241529569404204/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "changed": true,
    "invocation": {
        "module_args": {
            "api_version": "auto",
            "auto_remove": false,
            "blkio_weight": null,
            "ca_cert": null,
            "cap_drop": null,
            "capabilities": null,
            "cleanup": false,
            "client_cert": null,
            "client_key": null,
            "command": null,
            "comparisons": null,
            "cpu_period": null,
            "cpu_quota": null,
            "cpu_shares": null,
            "cpuset_cpus": null,
            "cpuset_mems": null,
            "debug": false,
            "detach": false,
            "device_read_bps": null,
            "device_read_iops": null,
            "device_write_bps": null,
            "device_write_iops": null,
            "devices": null,
          

### Register MineRL Gym Environment to RLlib

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

In [ ]:
r.stop()